# Setup

In [ ]:
import networkx as nx
from networkx.classes.function import path_weight
from networkx.algorithms import approximation as approx
import matplotlib.pyplot as plt
import random

In [ ]:
start = 0
n = 100

G = nx.complete_graph(n + 1)

random.seed(17)

for (u, v) in G.edges():
  G.edges[u,v]['weight'] = random.randint(1, 1000)

In [ ]:
def greedy_tsp(G, source=start):
  path = []
  current = start
  next = None

  for i in range(n):
    edge = min([e for e in G.edges(current) if e[1] not in path], key=lambda x:G.get_edge_data(x[0], x[1])['weight'])
    path.append(current)
    current = edge[1]

  path.append(current)
  path.append(start)
  return path 

In [ ]:
initial_path = greedy_tsp(G)

In [ ]:
def path_cost(graph:nx.Graph, path:list):
  full_path = path.copy()
  if full_path[0] != start:
    full_path.insert(0, start)
    full_path.append(start)
    
  return path_weight(graph, full_path, weight='weight')

In [ ]:
path_cost(G, initial_path)

3640

# Task 1

## Population Creation

In [ ]:
chromosome = initial_path[1: -1]
print(chromosome)

[90, 30, 14, 13, 97, 16, 3, 66, 64, 56, 26, 24, 54, 99, 74, 50, 87, 61, 86, 60, 31, 12, 95, 79, 82, 27, 49, 98, 55, 76, 29, 10, 44, 47, 5, 48, 65, 40, 11, 9, 51, 34, 19, 84, 8, 96, 100, 62, 94, 15, 7, 70, 80, 93, 23, 25, 52, 41, 78, 72, 4, 45, 38, 42, 21, 81, 39, 68, 88, 32, 1, 36, 35, 63, 33, 89, 43, 17, 28, 77, 75, 22, 67, 73, 6, 57, 83, 53, 58, 46, 69, 18, 91, 85, 71, 20, 37, 92, 59, 2]


In [ ]:
population = [chromosome]

for i in range(0, n-1):
  for j in range(i+1, n):
    new_chr = chromosome.copy()
    new_chr[i], new_chr[j] = new_chr[j], new_chr[i]
    population.append(new_chr)

pop_size = len(population)
print(f"Population size: {pop_size}")

Population size: 4951


In [ ]:
fitness = [0] * pop_size

for i in range(pop_size): fitness[i] = path_cost(G, population[i])

In [ ]:
r = random.randint(0, sum(fitness))

pop_subset= []

for i in range(0, pop_size):
  pop_subset.append(population[i])
  if sum(fitness[:i+1]) > r: break

subset_size = len(pop_subset)
print(f"Subset size: {subset_size}")

Subset size: 4177


## Chromosome crossover

In [ ]:
def create_child(p1:list, p2:list):
  m = len(p1)
  child = [0] * m
  left = random.randint(0, m-2)
  right = random.randint(left+1, m-1)

  pushover = 0

  for i in range(left, right+1):
    child[i] = p1[i]
  
  for i in list(range(right+1, m)) + list(range(left)):
    node_to_add = p2[ (i + pushover) % m ]
    while node_to_add in child:
      pushover += 1
      node_to_add = p2[ (i + pushover) % m ]
    child[i] = node_to_add

  return child

In [ ]:
def crossover(population, alpha_values=None, crossover_rate=.5):
  new_population = []
  A2 = list()

  for i in range(0, len(population)):
    p1, p2 = random.sample(population, 2)

    if random.random() < crossover_rate:
      c1 = create_child(p1, p2)
      if alpha_values is not None: 
        p1_alpha = alpha_values[population.index(p1)]
        p2_alpha = alpha_values[population.index(p2)]
        A2.append((p1_alpha, p2_alpha))
      
      c2 = create_child(p2, p1)
      if alpha_values is not None: 
        p1_alpha = alpha_values[population.index(p1)]
        p2_alpha = alpha_values[population.index(p2)]
        A2.append((p1_alpha, p2_alpha))
    else:
      c1 = p1
      c2 = p2
      if alpha_values is not None:
        A2.append(alpha_values[population.index(p1)])
        A2.append(alpha_values[population.index(p2)])



    new_population.append(c1)
    new_population.append(c2)

  new_pop_size = len(new_population)
  print(f"Number of children: {new_pop_size}")

  return new_population, A2

In [ ]:
def mutate(population, mutation_rate=.01):
  num_to_mutate = round(len(population) * mutation_rate)

  to_mutate = random.sample(range(0, len(population)), num_to_mutate)

  for m in to_mutate:
    i, j = random.sample(range(n), 2)
    mutant = population[m]
    mutant[i], mutant[j] = mutant[j], mutant[i]

In [ ]:
new_population, _ = crossover(pop_subset)
mutate(new_population)

Number of children: 8354


## Find the best one from population

In [ ]:
fitness = [0] * len(new_population)

for i in range(len(new_population)): 
  fitness[i] = path_cost(G, new_population[i])

min_value = min(fitness)
min_index = fitness.index(min_value)

baseline1 = new_population[min_index]

print(baseline1)
print(f"Baseline 1: {min_value}" )

[90, 30, 14, 13, 97, 16, 3, 66, 64, 56, 26, 24, 54, 99, 74, 50, 87, 61, 86, 60, 31, 12, 95, 79, 82, 27, 49, 98, 55, 59, 29, 10, 44, 47, 5, 48, 65, 40, 11, 9, 51, 34, 19, 84, 8, 96, 100, 62, 94, 15, 7, 70, 80, 93, 23, 25, 52, 41, 78, 72, 4, 45, 38, 42, 21, 81, 39, 68, 88, 32, 1, 36, 35, 63, 33, 89, 43, 17, 28, 77, 75, 22, 67, 73, 6, 57, 83, 53, 58, 46, 69, 18, 91, 85, 71, 20, 37, 92, 76, 2]
Baseline 1: 3295



# Task 2

In [ ]:
cycle = approx.simulated_annealing_tsp(G, "greedy", source=start, alpha=.01)
print(cycle)
baseline2 = path_cost(G, cycle)
print(f"Baseline 2: {baseline2}")

[0, 90, 30, 14, 13, 97, 16, 3, 66, 64, 56, 26, 24, 54, 99, 74, 50, 87, 61, 86, 60, 31, 12, 95, 79, 82, 27, 49, 98, 55, 76, 29, 10, 44, 47, 5, 48, 65, 40, 11, 9, 51, 34, 19, 84, 8, 96, 100, 62, 94, 15, 7, 70, 80, 93, 23, 25, 52, 41, 78, 72, 4, 45, 38, 42, 21, 81, 39, 68, 88, 32, 1, 36, 35, 63, 33, 89, 43, 17, 28, 77, 75, 22, 67, 73, 6, 57, 83, 53, 58, 46, 69, 18, 91, 85, 71, 20, 37, 92, 59, 2, 0]
Baseline 2: 3640


# Task 3

In [ ]:
T = list()
A = list()
F = list()

for a in range(10, 1001, 1):
  alpha = a / 1000
  tour = approx.simulated_annealing_tsp(G, "greedy", source=start, alpha=alpha)
  fitness = path_cost(G, tour)
  
  T.append(tour[1:-1])
  A.append(alpha)
  F.append(fitness)

In [ ]:
r = random.randint(0, sum(F))

for i in range(0, len(T)):
  if sum(F) < r: break
  max_f = max(F)
  max_f_i = F.index(max_f)
  del T[max_f_i]
  del A[max_f_i]
  del F[max_f_i]

In [ ]:
T2, A2 = crossover(T, alpha_values=A)
mutate(T2)

Number of children: 74


In [ ]:
F2 = list()

for t in T2:
   fitness = path_cost(G, t)
   F2.append(fitness)

fittest = min(F2)
i = F2.index(fittest)

print(f"Fittest Tour: {T2[i]}")
print(f"Tour Alpha: {A2[i]}")
print(f"Tour Fitness: {F2[i]}")

Fittest Tour: [90, 30, 14, 13, 97, 16, 3, 66, 64, 56, 26, 24, 54, 99, 74, 50, 87, 61, 86, 60, 31, 12, 95, 79, 82, 27, 49, 98, 55, 59, 29, 10, 44, 47, 5, 48, 65, 40, 11, 9, 51, 34, 19, 84, 8, 96, 100, 62, 94, 15, 7, 70, 80, 93, 23, 25, 52, 41, 78, 72, 4, 45, 38, 42, 21, 81, 39, 68, 88, 32, 1, 36, 35, 63, 33, 89, 43, 17, 28, 77, 75, 22, 67, 73, 6, 57, 83, 53, 58, 46, 69, 18, 91, 85, 71, 20, 37, 92, 76, 2]
Tour Alpha: 0.026
Tour Fitness: 3295


In [ ]:
# Just some testing 

best_path = T2[i].copy()
best_path.insert(0, start)
best_path.append(0)
new_path = approx.simulated_annealing_tsp(G, best_path, source=start, alpha=A2[i])

print(new_path)
print(path_cost(G, new_path))

[0, 90, 30, 14, 13, 97, 16, 3, 66, 64, 56, 26, 24, 54, 99, 74, 50, 87, 61, 86, 60, 31, 12, 95, 79, 82, 27, 49, 98, 55, 59, 29, 10, 44, 47, 5, 48, 65, 40, 11, 9, 51, 34, 19, 84, 8, 96, 100, 62, 94, 15, 7, 70, 80, 93, 23, 25, 52, 41, 78, 72, 4, 45, 38, 42, 21, 81, 39, 68, 88, 32, 1, 36, 35, 63, 33, 89, 43, 17, 28, 77, 75, 22, 67, 73, 6, 57, 83, 53, 58, 46, 69, 18, 91, 85, 71, 20, 37, 92, 76, 2, 0]
3295


# Task 4

Baseline 1 tour cost: 3295

Baseline 2 tour cost: 3640

Baseline 3 tour cost: 3295


GA seemed to work fine on it's own for this task. It has a lot of randomness involved so it is more that it happaned to stumble upon the answer more than anything. I'm sure that there is some tuning that can be done make it less random. For example, picking the tours with the lowest costs to then create a new population from. Then repeating this process until the the fitness doesn't change. SA wasn't much help. It returned the same tour as the greedy_tsp algorithm. In task 3, almost all the values for alpha returned the same tour except for one. This one different tour happened to also be the best tour and the same tour as in task 1. Applying GA to the population in task 3 didn't really give us a new or better tour, just returned the same tour as the best one. I wanted to see if NetworkX's SA for tsp could find a better tour if given the best tour and the alpha that gave me the best tour. It settled on the same tour given. 